In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import scipy.sparse as sp
import scipy.sparse.linalg as spl

# P9.1

![image](P9_1.png)

# P9.2

In [2]:
facebook = pd.read_csv(
    # Dataset from the SNAP database
    "https://snap.stanford.edu/data/facebook_combined.txt.gz",
    compression="gzip",
    sep=" ",
    names=["start_node", "end_node"],
)
G_facebook = nx.from_pandas_edgelist(facebook, "start_node", "end_node")

In [3]:
A = nx.adjacency_matrix(G_facebook)
k = np.array(A.sum(axis=1)).flatten()
pi = k / k.sum()
P = sp.diags(1 / k) @ A
evals, evecs = spl.eigs(P.T, k=1)
eigenvector = evecs[:,0].real
eigenvector = eigenvector / eigenvector.sum()

In [4]:
v = [d / (2 * G_facebook.size()) for n, d in G_facebook.degree()]
v = np.array(v)

In [5]:
np.linalg.norm(eigenvector - v)

np.float64(1.9435916528716124e-14)

# P9.3

In [45]:
df = pd.read_csv("https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book1-edges.csv")
G = nx.from_pandas_edgelist(df, "Source", "Target", edge_attr="weight")
nodes = list(G.nodes())

In [46]:
v_analytical = np.array([G.degree(n, weight="weight") for n in nodes])
v_analytical = v_analytical / (2 * G.size(weight="weight"))

In [47]:
A = nx.to_numpy_array(G, nodelist=nodes, weight="weight")
P = A / A.sum(axis=1, keepdims=True)
v_numerical = np.linalg.matrix_power(P, 1000)[0]

In [48]:
print(f"Match: {np.allclose(v_analytical, v_numerical)}")
print(f"Diff: {np.sum(np.abs(v_analytical - v_numerical)):.10f}")

Match: True
Diff: 0.0000000000
